In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("../")

In [3]:
import pandas as pd
import numpy as np
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

In [4]:
df = pd.read_parquet('/Users/rguarise/Documents/mlops-zoomcamp/data/yellow_tripdata_2022-01.parquet')

### Question 1: Number of columns in Jan 2022 Yellow Taxi Trip data

In [5]:
len(df.columns)

19

### Question 2:  Standard deviation of the trips duration in Jan 2022 Yellow Taxi Trip data

In [6]:
df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
np.std(df.duration)

46.445295712725304

### Question 3: Fraction of the records left after dropping the outliers

In [11]:
df_without_outliers = df[(df.duration >= 1) & (df.duration <= 60)].copy()

In [8]:
df_without_outliers.shape[0]/df.shape[0]

0.9827547930522406

### Question 4: Dimensionality after OHE

In [12]:
categorical = ['PULocationID', 'DOLocationID']
df_without_outliers[categorical] = df_without_outliers[categorical].astype(str)
train_dicts = df_without_outliers[categorical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

In [13]:
len(dv.feature_names_)

515

## Model

In [14]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
        df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [16]:
df_train = read_dataframe('/Users/rguarise/Documents/mlops-zoomcamp/data/yellow_tripdata_2022-01.parquet')
df_val = read_dataframe('/Users/rguarise/Documents/mlops-zoomcamp/data/yellow_tripdata_2022-02.parquet')

In [22]:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [23]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

### Question 5: RMSE on train

In [24]:
y_pred = lr.predict(X_train)
mean_squared_error(y_train, y_pred, squared=False)

6.986191065500608

### Question 6: RMSE on validation

In [25]:
y_pred = lr.predict(X_val)
mean_squared_error(y_val, y_pred, squared=False)

7.786408015215065